# 动态规划练习题

以下是一些动态规划的练习题，按照难度递增排列。每道题都包含问题描述和示例。

## 练习题 1: 最大子数组和

给定一个整数数组，找到具有最大和的连续子数组（子数组最少包含一个元素），返回其最大和。

**示例:**
```
输入: [-2,1,-3,4,-1,2,1,-5,4]
输出: 6
解释: 连续子数组 [4,-1,2,1] 的和最大，为 6。
```

**要求:** 使用动态规划解决，时间复杂度 O(n)，空间复杂度 O(1)。

In [5]:
def max_subarray(nums):
    """
    找到具有最大和的连续子数组
    """
    n = len(nums)
    dp = nums.copy() # dp[i]表示以i为结尾的子数组的最大和
    dp[0] = nums[0]
    max_sum = nums[0]
    max_indx = 0
    prev = [-1] * n
    
    
    for i in range(1, n):
        # 对每个元素遍历寻找最大和
        if dp[i - 1] + nums[i] > nums[i]:
            dp[i] = dp[i - 1] + nums[i]
            prev[i] = i - 1

        if dp[i] > max_sum:
            max_sum = dp[i]
            max_indx = i

    subarray = []
    i = max_indx
    while i != -1:
        subarray.append(nums[i])
        i = prev[i]
    return subarray, max_sum



In [7]:
print(max_subarray([-2,1,-3,4,-1,2,1,-5,4]))

([1, 2, -1, 4], 6)


## 练习题 2: 硬币找零

给定不同面额的硬币和一个总金额，计算可以凑成总金额所需的最少的硬币个数。如果没有任何一种硬币组合能组成总金额，返回 -1。

**示例:**
```
输入: coins = [1, 2, 5], amount = 11
输出: 3 
解释: 11 = 5 + 5 + 1
```

**要求:** 使用动态规划解决，时间复杂度 O(amount * len(coins))。

In [14]:
def coin_change_greedy(coins, amount):
    """
    计算最少硬币数（贪心算法版）
    """
    coins.sort(reverse=True)  # 从大到小排序
    count = 0
    for coin in coins:
        if amount == 0:
            break
        use = amount // coin  # 能用多少个当前面值的硬币
        count += use
        amount -= use * coin  # 减去已经用的部分
    if amount != 0:
        return -1  # 如果没法凑成金额，返回-1
    return count

In [ ]:
def coin_change_dp(coins, amount):
    """
    动态规划版
    """
    INF = float('inf')
    K = [INF] * (amount + 1)

    # 初始化
    K[0] = 0

    for i in range(1, amount + 1):
        for coin in coins:
            if i - coin >= 0: # coin得能用才行
                 K[i] = min(K[i - coin] + 1, K[i])

    if K[amount] == INF:
        return -1
    else:
        return K[amount]

In [15]:
coin_change_greedy(coins=[1, 2, 5], amount=11)

3

In [12]:
coin_change_dp(coins=[1, 2, 5], amount=11)

3

In [20]:
def coin_change_comb(coins, amount):
    """
    动态规划改进版：输出实际的最佳组合
    """
    INF = float('inf')
    K = [INF] * (amount + 1)
    prev = [-1] * (amount + 1)

    K[0] = 0

    for i in range(1, amount + 1):
        for coin in coins:
            if i >= coin and K[i - coin] + 1 < K[i]:
                    K[i] = K[i - coin] + 1
                    prev[i] = i - coin

    if K[amount] == INF:
         return -1
    
    comb = []
    curr = amount
    while prev[curr] != -1:
         comb.append(curr - prev[curr]) # coin = i - prev[i]
         curr = prev[curr]
    return comb

In [21]:
coin_change_comb(coins=[1, 2, 5], amount=11)

[1, 5, 5]

## 练习题 3: 不同路径

一个机器人位于一个 m x n 网格的左上角。机器人每次只能向下或者向右移动一步。机器人试图到达网格的右下角。问总共有多少条不同的路径？

**示例:**
```
输入: m = 3, n = 2
输出: 3
解释: 从左上角开始，总共有 3 条路径可以到达右下角。
1. 向右 -> 向右 -> 向下
2. 向右 -> 向下 -> 向右
3. 向下 -> 向右 -> 向右
```

**要求:** 使用动态规划解决，可以尝试空间优化。

In [24]:
def unique_paths(m, n):
    """
    计算不同路径数
    """
    # 初始化表格
    K = [[1] * n for _ in range(m)]

    # 从(1,1)开始
    for i in range(1, m):
        for j in range(1, n):
            K[i][j] = K[i - 1][j] + K[i][j - 1]
    
    return K[m - 1][n - 1]

In [25]:
unique_paths(3,2)

3

## 练习题 4-1: 戳气球

有 n 个气球，编号从 0 到 n-1，每个气球上都有一个数字，存储在数组 nums 中。你想要戳破所有的气球。如果你戳破气球 i ，你可以获得 nums[i-1] * nums[i] * nums[i+1] 枚硬币。如果 i-1 或 i+1 超出了数组的边界，那么就当它为 1。求你能获得的最大硬币数。

**示例:**
```
输入: [3,1,5,8]
输出: 167
解释: 
nums = [3,1,5,8] --> [3,5,8] --> [3,8] --> [8] --> []
coins = 3*1*5 + 3*5*8 + 1*3*8 + 1*8*1 = 167
```

**要求:** 使用区间动态规划解决。

In [16]:
def max_coins(nums):
    """
    计算能获得的最大硬币数
    这其实就是“最优二叉树搜索（OBST）问题”
    """
    data = [1] + nums + [1]
    n = len(data)
    dp = [[0] * n for _ in range(n)] # n * n的方阵

    # 初始条件是长度为1/0的情况，很显然此时收益为0，恰好与我们的初始值相同，因此无需理会
    
    # dp[i][j] = dp[i][k] + dp[k][j] + i * k * j,  i < k <j
    # 由于大区间的计算依赖于小区间，因此不能用嵌套for循环来计算
    # for i in range(n + 1):
    #     for j in range(i, n + 1):
    #         for k in range(i + 1, j):
    #             dp[i][j] = max(dp[i][k] + dp[k][j] + data[i] * data[k] * data[j])
    # 必须按区间长度来循环，由小到大逐步增大

    for length in range(2, n): # 区间长度为2 ~ n - 1
            for i in range(n - length): # 起始点为0 ~ n - length，防止越界
                j = i + length
                max_k_sum = 0
                for k in range(i + 1, j): # k的范围为i+1 ~ j-1
                    curr_k_sum = dp[i][k] + dp[k][j] + data[i] * data[k] * data[j]
                    if curr_k_sum > max_k_sum:
                            max_k_sum = curr_k_sum
                dp[i][j] = max_k_sum
                        
    return dp[0][n - 1] 

In [17]:
max_coins([3, 1, 5, 8])

167

## 练习题 4-2: 主题公园规划
有一个 n 个顶点的凸多边形，代表一块未开发的土地。每个顶点 i 上都有一个代表“人气值”的数字，存储在数组 pop 中。你的任务是将该多边形完全切分成若干个三角形区域。每形成一个由顶点 i, j, k 组成的三角形，你就可以获得 pop[i] * pop[j] * pop[k] 的商业价值。求你能获得的最大总商业价值。

**示例:**
```
输入: [10, 2, 5, 8]
输出: 500
解释: 
pop = [10, 2, 5, 8]
方案1 (连接顶点0和2): 三角形(0,1,2) + 三角形(0,2,3)
总价值 = (10*2*5) + (10*5*8) = 100 + 400 = 500

方案2 (连接顶点1和3): 三角形(0,1,3) + 三角形(1,2,3)
总价值 = (10*2*8) + (2*5*8) = 160 + 80 = 240

最大总价值为 500。
```

**要求:** 使用区间动态规划解决。

In [11]:
def park_max_value(pop):
    """
    使用区间动态规划解决凸多边形三角剖分最大价值问题
    """
    n = len(pop)  # 节点数量
    if n < 3:
        return 0
    
    # dp[i][j] 表示从顶点i到顶点j之间的多边形能获得的最大价值
    dp = [[0] * n for _ in range(n)]
    
    # length是区间长度，我们需要考虑所有可能的三角形剖分
    # 区间长度从3开始（最少3个点才能构成一个三角形）
    for length in range(3, n + 1): 
        # i是区间的起始点
        for i in range(n - length + 1):
            # j是区间的结束点
            j = i + length - 1
            
            # 在区间(i,j)中选择一个点k作为三角形的顶点
            # 该三角形由点i, k, j构成
            for k in range(i + 1, j):
                curr_val = dp[i][k] + dp[k][j] + pop[i] * pop[k] * pop[j]
                dp[i][j] = max(dp[i][j], curr_val)
    
    # 返回整个区间[0, n-1]的最大价值
    return dp[0][n-1]

In [12]:
park_max_value([10, 2, 5, 8])

500

## 练习题 4-3: 石子合并游戏

在一个直线上，有 n 堆石子排成一行。每堆石子都有一个代表“重量”的数字，存储在数组 `weights` 中。你的任务是将所有石子合并成一堆。每次操作，你只能选择**相邻**的两堆石子进行合并，合并后形成的新石子堆的重量是原来两堆石子重量之和。每次合并的“成本”也是这两堆石子重量之和。求将所有石子合并成一堆所需的最小总成本。

**示例:**
输入:

```python
[4, 1, 1, 4]
```

输出:

```
18
```

解释:
weights = \[4, 1, 1, 4]

我们来分析一种合并方式:

1. 先合并第2堆和第3堆 (1, 1)，成本为 1+1=2。石子堆变为 \[4, 2, 4]。
2. 接着合并新生成的第一堆和第二堆 (4, 2)，成本为 4+2=6。石子堆变为 \[6, 4]。
3. 最后合并剩下的两堆 (6, 4)，成本为 6+4=10。石子堆变为 \[10]。
   总成本 = 2 + 6 + 10 = 18。

我们再看另一种合并方式:

1. 先合并第1堆和第2堆 (4, 1)，成本为 4+1=5。石子堆变为 \[5, 1, 4]。
2. 再合并新生成的第二堆和第三堆 (1, 4)，成本为 1+4=5。石子堆变为 \[5, 5]。
3. 最后合并 (5, 5)，成本为 5+5=10。
   总成本 = 5 + 5 + 10 = 20。

对比发现，第一种方案的总成本 18 是最小的。

**要求:** 使用区间动态规划解决。

In [26]:
def stone_merge(weights):
    n = len(weights)
    INF = float('inf')
    dp = [[INF] * n for _ in range(n)]
    
    prefix_sum = [0] # 这道题需要大量的加和，用一个前缀和存储中间结果可以用空间换时间
    for w in weights:
        prefix_sum.append(prefix_sum[-1] + w)

    # 初始化 -- 相邻的两个合并成本为简单的加和
    for i in range(n - 1):
        j = i + 1
        dp[i][j] = weights[i] + weights[j]

    # 长度为0的
    for i in range(n):
        dp[i][i] = 0

    for length in range(3, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1

            for k in range(i, j):
                total = prefix_sum[j + 1] - prefix_sum[i]
                dp[i][j] = min(dp[i][j], dp[i][k] + dp[k + 1][j] + total)
    
    return dp[0][n - 1]


In [27]:
stone_merge([4, 1, 1, 4])

18

## 练习题 5: 编辑距离变种 - 最少操作次数

给定两个字符串 word1 和 word2，返回使 word1 和 word2 相同所需的最小步数。
每步可以删除任意一个字符串中的一个字符。

**示例:**
```
输入: word1 = "sea", word2 = "eat"
输出: 2
解释: 第一步将 "sea" 变为 "ea"，第二步将 "eat" 变为 "ea"
```

**要求:** 使用动态规划解决，这个问题与编辑距离类似但操作不同。

In [4]:
def min_distance(word1, word2):
    """
    计算使两个字符串相同的最小步数，
    本质上只需要计算最长公共子序列
    """
    m = len(word1)
    n = len(word2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # 初始化
    # dp[0][*], dp[*][0] 均为0

    for i in range (1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]: # 注意索引的位置，第i个字母为i-1
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    lcs = dp[m][n]

    return m + n - 2 * lcs

In [5]:
word1, word2 = "sea", "eat"
min_distance(word1, word2)

2

## 练习题 6: 最长回文子序列

给定一个字符串 s，找出其中最长的回文子序列的长度。
子序列是指在不改变字符顺序的前提下，通过删除一些字符（或不删除）得到的序列。

**示例:**
```
输入: s = "bbbab"
输出: 4
解释: 一个可能的最长回文子序列为 "bbbb"。
```

**要求:** 使用区间动态规划解决。

In [10]:
def longest_palindrome_subseq(s):
    """
    找出最长回文子序列的长度
    """
    n = len(s)

    dp = [[0] * (n + 1) for _ in range(n + 1)]

    # 初始化
    for i in range(n):
        dp[i][i] = 1
    
    # 主逻辑
    for length in range(1, n + 1): # 长度为1~n
        for i in range(n - length):
            j = i + length
            
            if s[i] == s[j]:
                dp[i][j] = 2 + dp[i + 1][j - 1]
            else:
                dp[i][j] = max(dp[i + 1][j], dp[i][j - 1])
    
    return dp[0][n - 1]


In [11]:
s = 'bbbab'
longest_palindrome_subseq(s)

4

In [ ]:
def get_longest_palindrome_subseq_path(s):
    """
    找出最长回文子序列的长度之外，输出实际的序列
    """
    n = len(s)
    
    dp = [[0] * n for _ in range(n)]
    dir = [[0] * n for _ in range(n)] # 导航图
    
    # 初始化
    for i in range(n):
        dp[i][i] = 1
    
    # 主逻辑
    for length in range(2, n + 1): # 长度为2~n
        for i in range(n - length + 1):
            j = i + length - 1
            
            if s[i] == s[j]:
                dp[i][j] = 2 + dp[i + 1][j - 1]
                dir[i][j] = 1 # 路标指向左下角
            else:
                dp[i][j] = max(dp[i + 1][j], dp[i][j - 1])
                if dp[i + 1][j] > dp[i][j - 1]:
                    dp[i][j] = dp[i + 1][j]
                    dir[i][j] = 2 # 路标指向下方
                else:
                    dp[i][j] = dp[i][j - 1]
                    dir[i][j] = 3 # 路标指向左边
    
    # 组装路径
    left_half = []
    right_half = []
    i = 0
    j = n - 1
    
    while i <= j:
        if i == j:
            left_half.append(s[i])
            break
        
        direction = dir[i][j]
        if direction == 1:
            left_half.append(s[i])
            right_half.append(s[j])
            i+=1
            j-=1
        elif direction == 2:
            i+=1
        else:
            j-=1
    
    path = "".join(left_half) + "".join(reversed(right_half))
    
    return dp[0][n - 1], path

In [ ]:
get_longest_palindrome_subseq_path('bbbab')

## 练习题 7: 分割等和子集



给定一个只包含正整数的非空数组，判断是否可以将这个数组分割成两个子集，使得两个子集的元素和相等。



**示例:**

```

输入: [1, 5, 11, 5]

输出: true

解释: 数组可以分割成 [1, 5, 5] 和 [11]。

```



**要求:** 这是一个0-1背包问题的变种，使用动态规划解决。

In [59]:
def can_patitioned(nums):
    """
    关键点在于，只需要选择一些数，使得和到达一半就可以了
    """
    # 可行性分析
    sums = sum(nums)
    if sums % 2 != 0:
        return False

    target = sums // 2
    n = len(nums)

    dp = [[False] * (target + 1) for _ in range(n + 1)] # dp[i][j] 表示用前i个数拼出j
    
    # 初始化
    for i in range(n):
        dp[i][0] = True

    # 主逻辑
    for i in range(1, n + 1): # 因为是实际的数字，不是索引，所以要加1
        num = nums[i - 1] # 当前数字
        for sub_target in range(target + 1):
            if num > sub_target:
                dp[i][sub_target] = dp[i - 1][sub_target]
            else:
                dp[i][sub_target] = dp[i - 1][sub_target - num] or dp[i - 1][sub_target]


    return dp[n][target]

In [62]:
nums = [1, 5, 11, 5, 2, 1, 1]
can_patitioned(nums)

True

In [61]:
nums = [1, 6, 11, 5]
can_patitioned(nums)

False